In [16]:
def cheker (supply, demand):
    return sum(supply) == sum(demand)

def fict (supply, demand):
    diff = abs(sum(supply) - sum(demand))
    if sum(supply) < sum(demand):
        supply.append(diff)
    else:
        demand.append(diff)

In [17]:
def north_west_corner (supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    if (cheker (supply, demand) == False):
        fict (supply, demand)
    
    num_suppliers = len(supply_copy)
    num_consumers = len(demand_copy)
    
    result = [[0] * num_consumers for _ in range(num_suppliers)]
    
    i, j = 0, 0
    
    while i < num_suppliers and j < num_consumers:
        
        min_val = min(supply_copy[i], demand_copy[j])
            
        result[i][j] = min_val
        
        supply_copy[i] -= min_val
        demand_copy[j] -= min_val
        
        if supply_copy[i] == 0:
            i += 1
        else:
            j += 1
    
    return result

In [18]:
def bring_task_to_close_type(matrix_cost, ai_vector_storage, bj_vector_shop):
    sum_ai = sum(ai_vector_storage)
    sum_bj = sum(bj_vector_shop)

    if sum_ai == sum_bj:
        return matrix_cost, ai_vector_storage, bj_vector_shop
    if sum_ai < sum_bj:
        ai_vector_storage.append(sum_bj - sum_ai)
        matrix_cost.append([0] * len(bj_vector_shop))
    else:
        bj_vector_shop.append(sum_ai - sum_bj)
        matrix_cost = [row + [0] for row in matrix_cost]

    return matrix_cost, ai_vector_storage, bj_vector_shop

def find_min_element_in_matrix(matrix, visited):
    min_value = float('inf')
    imin_row, jmin_col = 0, 0

    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if not visited[i][j] and matrix[i][j] < min_value:
                min_value = matrix[i][j]
                imin_row, jmin_col = i, j

    return imin_row, jmin_col

def find_base_solution_min_cost(matrix_cost, ai_vector_storage, bj_vector_shop):
    rows = len(ai_vector_storage)
    cols = len(bj_vector_shop)
    reference_basic = rows + cols - 1

    matrix_x_ij = [[None for _ in range(cols)] for _ in range(rows)]
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    record = 0

    while record < reference_basic:
        i, j = find_min_element_in_matrix(matrix_cost, visited)
        min_ai, min_bj = ai_vector_storage[i], bj_vector_shop[j]
        min_ai_bj = min(min_ai, min_bj)

        if min_ai_bj >= 0:
            matrix_x_ij[i][j] = min_ai_bj
            record += 1
            ai_vector_storage[i] -= min_ai_bj
            bj_vector_shop[j] -= min_ai_bj

        if ai_vector_storage[i] == 0:
            for col in range(cols):
                visited[i][col] = True
        if bj_vector_shop[j] == 0:
            for row in range(rows):
                visited[row][j] = True

    return matrix_x_ij

def print_matrix_x_ij(matrix_x_ij):
    print("Матрица перевозок (x_ij):")
    for row in matrix_x_ij:
        print(" ".join(str(x) if x is not None else "-" for x in row))

def method_potentials(matrix_cost, matrix_x_ij):
    rows = len(matrix_cost)
    cols = len(matrix_cost[0])

    ai_potential = [None] * rows
    bj_potential = [None] * cols

    ai_potential[0] = 0
    record_ai_potentials, record_bj_potentials = 1, 0

    while record_ai_potentials != len(ai_potential) or record_bj_potentials != len(bj_potential):
        for i in range(rows):
            for j in range(cols):
                if matrix_x_ij[i][j] is not None:
                    if ai_potential[i] is None and bj_potential[j] is not None:
                        ai_potential[i] = matrix_cost[i][j] - bj_potential[j]
                        record_ai_potentials += 1
                    if ai_potential[i] is not None and bj_potential[j] is None:
                        bj_potential[j] = matrix_cost[i][j] - ai_potential[i]
                        record_bj_potentials += 1

    return ai_potential, bj_potential

def get_first_non_basic_or_optimal(matrix_x_ij, ai_potential, bj_potential, matrix_cost):
    for i in range(len(matrix_x_ij)):
        for j in range(len(matrix_x_ij[i])):
            if matrix_x_ij[i][j] is None and ai_potential[i] + bj_potential[j] > matrix_cost[i][j]:
                return i, j
    return True

def find_closed_loop(matrix_x_ij, i_start, j_start):
    stack = [((i_start, j_start), None, [])]
    visited = set()

    while stack:
        current, direction, path = stack.pop()
        if current == (i_start, j_start) and len(path) >= 4:
            return path

        if current in visited and current != (i_start, j_start):
            continue

        path.append(current)
        visited.add(current)

        neighbors = []
        if direction is None:
            for col in range(len(matrix_x_ij[0])):
                if matrix_x_ij[current[0]][col] is not None:
                    neighbors.append(((current[0], col), "col"))
            for row in range(len(matrix_x_ij)):
                if matrix_x_ij[row][current[1]] is not None:
                    neighbors.append(((row, current[1]), "row"))
        if direction == "row":
            for col in range(len(matrix_x_ij[0])):
                if (matrix_x_ij[current[0]][col] is not None or
                        (current[0] == i_start and col == j_start and len(path) > 3)):
                    neighbors.append(((current[0], col), "col"))
        if direction == "col":
            for row in range(len(matrix_x_ij)):
                if (matrix_x_ij[row][current[1]] is not None or
                        (row == i_start and current[1] == j_start and len(path) > 3)):
                    neighbors.append(((row, current[1]), "row"))

        for neighbor, next_direction in neighbors:
            if neighbor not in visited or neighbor == (i_start, j_start):
                stack.append((neighbor, next_direction, path.copy()))

    return None

def recalculate_cycle(matrix_x_ij, path):
    minimal = float('inf')
    for row, col in path[1::2]:
        current_value = matrix_x_ij[row][col]
        if current_value is not None and current_value > 0:
            minimal = min(minimal, current_value)

    for row, col in path[0::2]:
        if matrix_x_ij[row][col] is None:
            matrix_x_ij[row][col] = 0
        matrix_x_ij[row][col] += minimal

    for row, col in path[1::2]:
        matrix_x_ij[row][col] -= minimal
        if matrix_x_ij[row][col] == 0:
            matrix_x_ij[row][col] = None

def calculate_optimal_value_and_print_matrixs(matrix_cost, matrix_x_ij):
    L_star = 0
    for i in range(len(matrix_cost)):
        for j in range(len(matrix_cost[i])):
            if matrix_x_ij[i][j] is not None:
                L_star += matrix_cost[i][j] * matrix_x_ij[i][j]
    print_matrix_x_ij(matrix_x_ij)
    print(f'\nЗначение функции L = {L_star}')

def solve_transport_task(matrix_cost, ai_vector_storage, bj_vector_shop):
    matrix_cost, ai_vector_storage, bj_vector_shop = bring_task_to_close_type(matrix_cost, ai_vector_storage, bj_vector_shop)
    matrix_x_ij = find_base_solution_min_cost(matrix_cost, ai_vector_storage, bj_vector_shop)

    while True:
        ai_potential, bj_potential = method_potentials(matrix_cost, matrix_x_ij)
        result = get_first_non_basic_or_optimal(matrix_x_ij, ai_potential, bj_potential, matrix_cost)

        if result is True:
            calculate_optimal_value_and_print_matrixs(matrix_cost, matrix_x_ij)
            break
        else:
            i_start, j_start = result
            path = find_closed_loop(matrix_x_ij, i_start, j_start)
            if path is None:
                raise KeyError
            recalculate_cycle(matrix_x_ij, path)

# Пример использования
matrix_cost = [[21, 19, 11, 12, 12], [26, 29, 14, 1, 26], [39, 1, 22, 8, 25], [53, 20, 40, 26, 28]]
a_i = [24, 12, 18, 16]
b_j = [11, 13, 26, 10, 10]

solve_transport_task(matrix_cost, a_i, b_j)

Матрица перевозок (x_ij):
11 - 13 - -
- - 12 - -
- 13 - 5 -
- - 1 5 10

Значение функции L = 1045


In [19]:
def bring_task_to_close_type(matrix_cost, ai_vector_storage, bj_vector_shop):
    sum_ai = sum(ai_vector_storage)
    sum_bj = sum(bj_vector_shop)

    if sum_ai == sum_bj:
        return matrix_cost, ai_vector_storage, bj_vector_shop
    if sum_ai < sum_bj:
        ai_vector_storage.append(sum_bj - sum_ai)
        matrix_cost.append([0] * len(bj_vector_shop))
    else:
        bj_vector_shop.append(sum_ai - sum_bj)
        matrix_cost = [row + [0] for row in matrix_cost]

    return matrix_cost, ai_vector_storage, bj_vector_shop

In [20]:
def find_base_solution_north_west(matrix_cost, ai_vector_storage, bj_vector_shop):
    rows = len(ai_vector_storage)
    cols = len(bj_vector_shop)
    matrix_x_ij = [[None for _ in range(cols)] for _ in range(rows)]

    i, j = 0, 0

    while i < rows and j < cols:
        min_ai_bj = min(ai_vector_storage[i], bj_vector_shop[j])
        matrix_x_ij[i][j] = min_ai_bj
        ai_vector_storage[i] -= min_ai_bj
        bj_vector_shop[j] -= min_ai_bj

        if ai_vector_storage[i] == 0:
            i += 1
        if bj_vector_shop[j] == 0:
            j += 1

    return matrix_x_ij
    
    

In [21]:
def method_potentials(matrix_cost, matrix_x_ij):
    rows = len(matrix_cost)
    cols = len(matrix_cost[0])

    ai_potential = [None] * rows
    bj_potential = [None] * cols

    ai_potential[0] = 0
    record_ai_potentials, record_bj_potentials = 1, 0

    while record_ai_potentials != len(ai_potential) or record_bj_potentials != len(bj_potential):
        for i in range(rows):
            for j in range(cols):
                if matrix_x_ij[i][j] is not None:
                    if ai_potential[i] is None and bj_potential[j] is not None:
                        ai_potential[i] = matrix_cost[i][j] - bj_potential[j]
                        record_ai_potentials += 1
                    if ai_potential[i] is not None and bj_potential[j] is None:
                        bj_potential[j] = matrix_cost[i][j] - ai_potential[i]
                        record_bj_potentials += 1

    return ai_potential, bj_potential

In [22]:
def get_first_non_basic_or_optimal(matrix_x_ij, ai_potential, bj_potential, matrix_cost):
    for i in range(len(matrix_x_ij)):
        for j in range(len(matrix_x_ij[i])):
            if matrix_x_ij[i][j] is None and ai_potential[i] + bj_potential[j] > matrix_cost[i][j]:
                return i, j
    return True

In [23]:
def find_closed_loop(matrix_x_ij, i_start, j_start):
    stack = [((i_start, j_start), None, [])]
    visited = set()

    while stack:
        current, direction, path = stack.pop()
        if current == (i_start, j_start) and len(path) >= 4:
            return path

        if current in visited and current != (i_start, j_start):
            continue

        path.append(current)
        visited.add(current)

        neighbors = []
        if direction is None:
            for col in range(len(matrix_x_ij[0])):
                if matrix_x_ij[current[0]][col] is not None:
                    neighbors.append(((current[0], col), "col"))
            for row in range(len(matrix_x_ij)):
                if matrix_x_ij[row][current[1]] is not None:
                    neighbors.append(((row, current[1]), "row"))
        if direction == "row":
            for col in range(len(matrix_x_ij[0])):
                if (matrix_x_ij[current[0]][col] is not None or
                        (current[0] == i_start and col == j_start and len(path) > 3)):
                    neighbors.append(((current[0], col), "col"))
        if direction == "col":
            for row in range(len(matrix_x_ij)):
                if (matrix_x_ij[row][current[1]] is not None or
                        (row == i_start and current[1] == j_start and len(path) > 3)):
                    neighbors.append(((row, current[1]), "row"))

        for neighbor, next_direction in neighbors:
            if neighbor not in visited or neighbor == (i_start, j_start):
                stack.append((neighbor, next_direction, path.copy()))

    return None

In [24]:
def recalculate_cycle(matrix_x_ij, path):
    minimal = float('inf')
    for row, col in path[1::2]:
        current_value = matrix_x_ij[row][col]
        if current_value is not None and current_value > 0:
            minimal = min(minimal, current_value)

    for row, col in path[0::2]:
        if matrix_x_ij[row][col] is None:
            matrix_x_ij[row][col] = 0
        matrix_x_ij[row][col] += minimal

    for row, col in path[1::2]:
        matrix_x_ij[row][col] -= minimal
        if matrix_x_ij[row][col] == 0:
            matrix_x_ij[row][col] = None

In [25]:
def print_matrix_x_ij(matrix_x_ij):
    print("Матрица перевозок (x_ij):")
    for row in matrix_x_ij:
        print("\t ".join(str(x) if x is not None else "-" for x in row))

In [26]:
def calculate_optimal_value_and_print_matrixs(matrix_cost, matrix_x_ij):
    L_star = 0
    for i in range(len(matrix_cost)):
        for j in range(len(matrix_cost[i])):
            if matrix_x_ij[i][j] is not None:
                L_star += matrix_cost[i][j] * matrix_x_ij[i][j]
    print_matrix_x_ij(matrix_x_ij)
    print(f'\nЗначение функции L = {L_star}')

In [27]:
def solve_transport_task(matrix_cost, ai_vector_storage, bj_vector_shop):
    matrix_cost, ai_vector_storage, bj_vector_shop = bring_task_to_close_type(matrix_cost, ai_vector_storage, bj_vector_shop)
    matrix_x_ij = find_base_solution_north_west(matrix_cost, ai_vector_storage, bj_vector_shop)

    while True:
        ai_potential, bj_potential = method_potentials(matrix_cost, matrix_x_ij)
        result = get_first_non_basic_or_optimal(matrix_x_ij, ai_potential, bj_potential, matrix_cost)

        if result is True:
            calculate_optimal_value_and_print_matrixs(matrix_cost, matrix_x_ij)
            break
        else:
            i_start, j_start = result
            path = find_closed_loop(matrix_x_ij, i_start, j_start)
            if path is None:
                raise KeyError
            recalculate_cycle(matrix_x_ij, path)

In [28]:
import numpy as np
matrix = np.array([[40, 36, 9, 20], [26, 11, 22, 26], [6, 3, 12, 3], [5, 37, 33, 26]])
supply = [24, 42, 23, 36]
demand = [35, 29, 21, 35]
result = solve_transport_task(matrix, supply, demand)


IndexError: list index out of range